In [1]:
#Import packages
import pandas as pd
pd.set_option('display.max_columns', None)

In [7]:
raw_data = pd.read_csv('Data/house_data_details_scraped.csv')
df = raw_data.copy()
print(df.shape)

(21491, 25)


In [8]:
df.head()

,latitude,longitude,address,property_type,rooms,home_size,lot_size,year_built,parcel_number,realtyID,county,subdivision,census,tract,lot,zoning,date,sale_price,estimated_value,sex_offenders,crime_index,enviornmental_hazards,natural_disasters,school_quality,url
0,33.974622,-118.133679,6224 Nye St,Condominium,"3 beds, 3 baths","1,357 sqft",0.51 acres,1988.0,6357012043,1.111054e+09,Los Angeles,45351,1.0,532304.0,1,\nZoning:\n\nCMR3\n\t\t,9/28/20,"$469,000","$439,000",7 Found,High,15 Found,1 Found,Average,https://www.realtytrac.com/property/ca/commerc...
1,34.156562,-118.396891,4723 Laurel Canyon Blvd,Restaurant,NaN,"2,250 sqft","9,799 sqft",1959.0,2356037034,1.111055e+09,Los Angeles,7360,1.0,143400.0,10,\nZoning:\n\nLAC2\n\t\t,9/28/20,"$1,910,000",NaN,5 Found,NaN,6 Found,1 Found,Excellent,https://www.realtytrac.com/property/ca/valley-...
2,34.677777,-118.451165,18118 Elizabeth Lake Rd,Apartment house (5+ units),"7 beds, 6 baths","1,454 sqft","13,635 sqft",1948.0,3242015023,1.111056e+09,Los Angeles,NaN,2.0,920102.0,4,\nZoning:\n\nLCC4*\n\t\t,9/28/20,"$325,000",NaN,0 Found,NaN,2 Found,1 Found,NaN,https://www.realtytrac.com/property/ca/lake-hu...
3,34.072960,-118.066898,9259 Ramona Blvd,Single Family Residence,"3 beds, 2 baths","1,682 sqft","7,000 sqft",1978.0,8594027016,1.111056e+09,Los Angeles,ROSEMEAD,2.0,432901.0,6,\nZoning:\n\nRMPOD*\n\t\t,9/28/20,"$738,000","$752,000",3 Found,Moderate,11 Found,1 Found,Excellent,https://www.realtytrac.com/property/ca/rosemea...
4,33.777716,-118.154913,825 Obispo Ave,"Triplex (3 units, any combination)","4 beds, 3 baths","1,958 sqft","6,754 sqft",1938.0,7258013016,1.111056e+09,Los Angeles,1,2.0,576904.0,18,\nZoning:\n\nLBR2N\n\t\t,9/28/20,"$1,185,000","$989,000",11 Found,Slightly High,8 Found,1 Found,Average,https://www.realtytrac.com/property/ca/long-be...


In [9]:
#Parse the bedroom feature
df['bedrooms'] = df['rooms'].apply(lambda x: x.split(',')[0] if x!=None and type(x)== str else None)
df['bedrooms'] = df['bedrooms'].replace('\D', '', regex=True)



#Parse the bathrooms feature
df['bathrooms'] = df['rooms'].apply(lambda x: x.split(',')[1] if x!=None and type(x)== str and x != 'Built in' else None)
df['bathrooms'] = df['bathrooms'].replace('\D', '', regex=True)

#Drop the Rooms column
df.drop('rooms', inplace=True, axis=1)

df[['bedrooms', 'bathrooms']][:5]

,bedrooms,bathrooms
0,3,3
1,None,None
2,7,6
3,3,2
4,4,3


In [10]:
#Parse the home unit
df['home_unit'] = df['home_size'].apply(lambda x: x.split(' ')[1] if x!=None and type(x)==str else None)

#Parse the home size
df['home_size_temp'] = df['home_size'].apply(lambda x: x if x!=None else None)
df.loc[df['home_unit']=='acres', ['home_size_temp']] = df['home_size'].apply(lambda x: x.split(' ')[0] if x!=None and type(x)== str else None).replace('\D', '', regex=True)
df.loc[df['home_unit']=='sqft', ['home_size_temp']] = df['home_size'].apply(lambda x: x.split(' ')[0] if x!=None and type(x)== str else None).replace('\D', '', regex=True)
df['home_size'] = df['home_size_temp']

#Drop the temp column
df.drop(['home_size_temp'], inplace=True, axis=1)

df[['home_size', 'home_unit']][:5]

,home_size,home_unit
0,1357,sqft
1,2250,sqft
2,1454,sqft
3,1682,sqft
4,1958,sqft


In [11]:
#Parse the lot unit
df['lot_unit'] = df['lot_size'].apply(lambda x: x.split(' ')[1] if x!=None and type(x)==str else None)

#Parse the lot size
df['lot_size_temp'] = df['lot_size'].apply(lambda x: x if x!=None else None)
df.loc[df['lot_unit']=='acres', ['lot_size_temp']] = df['lot_size'].apply(lambda x: x.split(' ')[0] if x!=None and type(x)== str else None).replace('\D', '', regex=True)
df.loc[df['lot_unit']=='sqft', ['lot_size_temp']] = df['lot_size'].apply(lambda x: x.split(' ')[0] if x!=None and type(x)== str else None).replace('\D', '', regex=True)
df['lot_size'] = df['lot_size_temp']

#Drop the temp and unit columns
df.drop(['lot_size_temp'], inplace=True, axis=1)

df[['lot_size', 'lot_unit']][:5]

,lot_size,lot_unit
0,051,acres
1,9799,sqft
2,13635,sqft
3,7000,sqft
4,6754,sqft


In [12]:
#Convert the acres into sqft
#First convert the data type to float
df[['home_size', 'lot_size']] = df[['home_size', 'lot_size']].astype(float)

df.loc[df['home_unit']=='acres', 'home_size'] = df['home_size']*43560.0
df.loc[df['lot_unit']=='acres', 'lot_size'] = df['lot_size']*43560.0

#Drop the unit columns
df.drop(['home_unit', 'lot_unit'], inplace=True, axis=1)

df[['home_size', 'lot_size']][:5]

,home_size,lot_size
0,1357.0,2221560.0
1,2250.0,9799.0
2,1454.0,13635.0
3,1682.0,7000.0
4,1958.0,6754.0


In [13]:
#Parse and clean Zoning
df['zoning'] = df['zoning'].replace('\s', '', regex=True).replace('Zoning:', '', regex=True).replace('\*?', '', regex=True)
df[['zoning']][:5]

,zoning
0,CMR3
1,LAC2
2,LCC4
3,RMPOD
4,LBR2N


In [14]:
#Parse out the sale price and estimated value
df['sale_price'] = df['sale_price'].replace('\D', '', regex=True)
df['estimated_value'] = df['estimated_value'].replace('\D', '', regex=True)
df[['sale_price', 'estimated_value']][:5]

,sale_price,estimated_value
0,469000,439000
1,1910000,NaN
2,325000,NaN
3,738000,752000
4,1185000,989000


In [15]:
#Parse out sex_offenders, enviornmental_hazards, natural_disasters
df[['sex_offenders', 'enviornmental_hazards', 'natural_disasters']] = df[['sex_offenders', 'enviornmental_hazards', 'natural_disasters']].replace('\D', '', regex=True)
df[['sex_offenders', 'enviornmental_hazards', 'natural_disasters']][:5]

,sex_offenders,enviornmental_hazards,natural_disasters
0,7,15,1
1,5,6,1
2,0,2,1
3,3,11,1
4,11,8,1


In [16]:
#Make sure each column has the right data type
df.dtypes

latitude                 float64
longitude                float64
address                   object
property_type             object
home_size                float64
lot_size                 float64
year_built               float64
parcel_number              int64
realtyID                 float64
county                    object
subdivision               object
census                   float64
tract                    float64
lot                       object
zoning                    object
date                      object
sale_price                object
estimated_value           object
sex_offenders             object
crime_index               object
enviornmental_hazards     object
natural_disasters         object
school_quality            object
url                       object
bedrooms                  object
bathrooms                 object
dtype: object

In [17]:
#All of the features that are to be numeric
numeric = ['sale_price', 'estimated_value', 'bedrooms', 'bathrooms', 'sex_offenders',  
           'enviornmental_hazards', 'natural_disasters']
date = ['date']
objects = ['parcel_number', 'realtyID']

In [18]:
#Convert home_size and lot_size to numeric types
df[numeric] = df[numeric].apply(pd.to_numeric)

#Convert the date of the sale price to date_time
df[date] = df[date].apply(pd.to_datetime)

#Convert the parcel_number and realtyid features into strings
df[objects] = df[objects].astype(str)

In [19]:
#Ensure everything is in order
df.dtypes

latitude                        float64
longitude                       float64
address                          object
property_type                    object
home_size                       float64
lot_size                        float64
year_built                      float64
parcel_number                    object
realtyID                         object
county                           object
subdivision                      object
census                          float64
tract                           float64
lot                              object
zoning                           object
date                     datetime64[ns]
sale_price                      float64
estimated_value                 float64
sex_offenders                     int64
crime_index                      object
enviornmental_hazards             int64
natural_disasters                 int64
school_quality                   object
url                              object
bedrooms                        float64


In [20]:
#Fix the geolocations for those that are reversed
proper_lat = df.loc[df['latitude']<33, 'longitude']
proper_long = df.loc[df['longitude']>-116, 'latitude']

df.loc[df['latitude']<33, 'latitude'] = proper_lat
df.loc[df['longitude']>-116, 'longitude'] = proper_long

In [21]:
#Remove any properties in the Catalina Island
print(df.shape)
df = df.loc[df['latitude']>33.6]
print(df.shape)

(21491, 26)
(21034, 26)


In [22]:
print(df.shape)
df = df.drop_duplicates(subset='parcel_number')
print(df.shape)

(21034, 26)
(17855, 26)


In [23]:
#Save the file
df.to_csv('Data/house_data_details_cleaned.csv', index=False)